In [9]:
%config IPCompleter.greedy=True

In [10]:
import pandas as pd
import numpy as np
from spotlight.factorization.implicit import ImplicitFactorizationModel
from spotlight.interactions import Interactions
from spotlight.cross_validation import random_train_test_split

from collections import defaultdict
from itertools import count

In [11]:
ratings_df = pd.read_csv('C:/Users/papad/Desktop/ml-latest-small/ml-latest-small/ratings.csv')
movies_df = pd.read_csv('C:/Users/papad/Desktop/ml-latest-small/ml-latest-small/movies.csv')
tags_df = pd.read_csv('C:/Users/papad/Desktop/ml-latest-small/ml-latest-small/tags.csv')
links_df = pd.read_csv('C:/Users/papad/Desktop/ml-latest-small/ml-latest-small/links.csv')

In [12]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [13]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


The ids are not consecutive. For examples the id of a movies is shown multiple times. For this reason the procedure below is implemented. The reversed map is still saved to be used if needed.

In [14]:
uid_map = defaultdict(count().__next__)
iid_map = defaultdict(count().__next__)
uids = np.array([uid_map[uid] for uid in ratings_df["userId"].values ], dtype=np.int32)
iids = np.array([iid_map[iid] for iid in ratings_df["movieId"].values ], dtype=np.int32)

uid_rev_map = {v: k for k, v in uid_map.items()}
iid_rev_map = {v: k for k, v in iid_map.items()}

ratings = ratings_df["rating"].values.astype(np.float32)
timestamps = ratings_df["timestamp"].values.astype(np.int32)

print("userId %d got uid %d" % (556, uid_map[556]))
print("movieId %d got iid %d" % (54001, iid_map[54001]))

userId 556 got uid 555
movieId 54001 got iid 2518


create the dataset variable representing the dataset prepared for use to create a recommendation systems using the Spotlight platform.

In [15]:
from spotlight.interactions import Interactions
from spotlight.cross_validation import random_train_test_split

explicit_interaction = Interactions(user_ids=uids,
                                  item_ids=iids,
                                  ratings=ratings,
                                  timestamps=timestamps)

implicit_interaction = Interactions(user_ids=uids,
                                   item_ids=iids)


#lets initialise the seed, so that its repeatable and reproducible 
explicit_train, explicit_test = random_train_test_split(explicit_interaction, random_state=np.random.seed(42))
implicit_train, implicit_test = random_train_test_split(implicit_interaction, random_state=np.random.seed(42))

Create an Explicit model 

In [16]:
from spotlight.factorization.explicit import ExplicitFactorizationModel
import time  

explicit_model = ExplicitFactorizationModel(n_iter=5,
                                    embedding_dim=32*4,#this is Spotlight default
                                    random_state=np.random.RandomState(1),
                                    use_cuda=False)
current = time.time()

explicit_model.fit(train, verbose=True)

end = time.time()
diff = end - current
print("Training took %d seconds "% (diff))

NameError: name 'train' is not defined

In [ ]:
from spotlight.evaluation import mrr_score

print(mrr_score(explicit_model, test, train=train, k=100).mean())

Create an Implicit model

In [ ]:
from spotlight.factorization.implicit import ImplicitFactorizationModel

implicit_model  = ImplicitFactorizationModel()
implicit_model.fit(implicit_train)
implicit_pred = implicit_model.predict(user_ids,item_ids = None)